## OpenWeather API - Ejercicios

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import plotly.express as px
import folium

import requests

from datetime import datetime
from time import sleep

from pprint import pprint

In [ ]:
def get_current_weather(lat, lon, api_key):
    
    params = {"appid" : api_key,
              "lat"   : lat,
              "lon"   : lon,
              "units" : "metric"}

    endpoint = f"https://api.openweathermap.org/data/2.5/weather"

    response = requests.get(url = endpoint, params = params)

    return response.json()

In [ ]:
api_key = "cd31574dc65a379909d4f921033a7e96"

### Ejercicio 01:

- Usando la función _**get_current_weather()**_ extrae la información principal de 10 países.
- Muestra la información en un mapa de **folium**.
- Muestra la diferencia de temperatura en un mapa **Choropleth** de esos 10 países.

In [ ]:
# Info

datos = [["Spain"      , 40.4637, -3.7492],
         ["Portugal"   , 39.3999, -8.2245],
         ["England"    , 55.3781, -3.4360],
         ["Germany"    , 51.1657, 10.4515],
         ["France"     , 46.2276, 2.2137 ],
         ["Italy"      , 41.8719, 12.5674],
         ["Greece"     , 39.0742, 21.8243],
         ["Netherlands", 52.1326, 5.2913 ],
         ["Swiss"      , 46.8182, 8.2275 ],
         ["Sweden"     , 60.1282, 18.6435]]

weather_data = list()

for pais, lat, lon in datos:
    
    data = get_current_weather(lat = lat, lon = lon, api_key = api_key)
    
    description = data["weather"][0]["description"]
    temp        = data["main"]["temp"]
    feels_like  = data["main"]["feels_like"]
    temp_min    = data["main"]["temp_min"]
    temp_max    = data["main"]["temp_max"]
    pressure    = data["main"]["pressure"]
    humidity    = data["main"]["humidity"]
    wind_speed  = data["wind"]["speed"]
    dt          = data["dt"]
    lat         = data["coord"]["lat"]
    lon         = data["coord"]["lon"]
    name        = data["name"]

    weather_data.append([pais, description, temp, feels_like, temp_min, 
                         temp_max, pressure, humidity, wind_speed, dt, lat, lon])
    
df_weather = pd.DataFrame(data    = weather_data,
                          columns = ["pais", "description", "temp", "feels_like", "temp_min", "temp_max",
                                     "pressure", "humidity", "wind_speed", "dt", "lat", "lon"])

In [ ]:
df_weather

In [ ]:
# Mapa folium

weather_map = folium.Map(location = [df_weather["lat"].mean(), df_weather["lon"].mean()], zoom_start = 3)

info = folium.map.FeatureGroup()

for lat, lng, pais, description, temp, pressure, humidity, wind_speed in zip(df_weather["lat"], df_weather["lon"],
                                                                             df_weather["pais"], df_weather["description"],
                                                                             df_weather["temp"], df_weather["pressure"],
                                                                             df_weather["humidity"], df_weather["wind_speed"]):
    
    string = f"Country: {pais}\nDescription: {description}\nTemp: {temp}\nPressure: {pressure}\nHumidity: {humidity}\nWind Speed: {wind_speed}"
    
    info.add_child(folium.Marker(location = [lat, lng],
                                 popup    = string))
    
weather_map.add_child(info)

weather_map

In [ ]:
# Choropleth

world_geo = "data/world_countries.json" # Archivo GeoJSON

# Crea un mapa mundial 
world_map = folium.Map(location = [df_weather["lat"].mean(), df_weather["lon"].mean()], zoom_start = 3)

folium.Choropleth(geo_data     = "data/world_countries.json",
                  data         = df_weather,
                  columns      = ["pais", "temp"],
                  key_on       = "feature.properties.name",
                  fill_color   = "Reds", 
                  fill_opacity = 0.7, 
                  line_opacity = 0.2,
                  legend_name  = "Temp").add_to(world_map)

world_map

### Ejercicio 02:
- Define una función que extraiga la información de contaminación del aire para un punto geográfico.
- La función debe de tomar los parámetros: _**start**_, _**end**_, _**api_key**_, _**lat**_ & _**lon**_.
- La función debe retornar un **DataFrame** y la columna de tiempo debe de ser un objeto **datetime**.

In [ ]:
def get_historical_air_pollution(start, end, api_key, lat, lon):
    
    params = {"appid" : api_key,
              "lat"   : lat,
              "lon"   : lon,
              "start" : start,
              "end"   : end}

    endpoint = "http://api.openweathermap.org/data/2.5/air_pollution/history"

    response = requests.get(url = endpoint, params = params)

    data = response.json()
    
    df_air = pd.json_normalize(data["list"])

    df_air.rename(mapper  = {x : x.split(".")[1] for x in df_air.columns[1:]},
                  axis    = 1,
                  inplace = True)
    
    df_air["dt"] = df_air["dt"].apply(lambda x : datetime.fromtimestamp(x))

    return df_air

### Ejercicio 03:

- Usa la función definida en el ejercicio anterior para extraer la información desde el año 2020 hasta el día de hoy de una ciudad.
- Realiza visualizaciones con estos datos. Prueba agrupando por mes, trimestre o año.

In [ ]:
%%time

lat, lon = 40.416775, -3.703790 # Madrid, Spain

start = int(datetime(2020, 1, 1).timestamp()) # 2020-01-01

end = int(datetime.now().timestamp()) # Hoy

df_air = get_historical_air_pollution(start = start, end = end, api_key = api_key, lat = lat, lon = lon)
    
df_air.shape

In [ ]:
df_air

In [ ]:
px.line(data_frame = df_air,
        x          = "dt",
        y          = "no2")

In [ ]:
df_air["year"] = df_air["dt"].apply(lambda x : x.year)

df_air["aqi"] = df_air["aqi"].astype(str)

In [ ]:
px.box(data_frame = df_air,
       x          = "aqi",
       y          = "no2",
       facet_col  = "year")

In [ ]:
# Bubble Chart

px.scatter(data_frame = df_air,
           x          = "no2",
           y          = "co",
           color      = "aqi",
           size       = "pm10",
           facet_row  = "year")

In [ ]:
px.scatter_3d(data_frame = df_air,
              x          = "no2",
              y          = "co",
              z          = "nh3",
              color      = "aqi",
              size       = "pm10")

In [ ]:
################################################################################################################################